In [1]:
from ultralytics import YOLO
import numpy as np
import cv2
import math
import time
import csv
from geopy.distance import geodesic
import os
from collections import defaultdict
from collections import deque



In [2]:
# CAmea Details
FOCAL_LENGTH_MM = 21
SENSOR_WIDTH_MM = 11
SENSOR_HEIGHT_MM = 8.3

In [3]:
# Drone GPS 
drone_lon = 82.984324
drone_lat = 25.263924
drone_alt = 6.706

In [4]:
# Drone IMU
imu_roll = 0.0
imu_pitch = 0.0
imu_yaw = 0.0

In [5]:
# Gimbal oriantation
gimbal_roll = 0
gimbal_pitch = -25.0
gimbal_yaw =0

In [6]:
# Feild of view in radians
def calculate_fov(sensor, focal):
    return 2*np.arctan(sensor/(2*focal))

fov_x = calculate_fov(SENSOR_WIDTH_MM,FOCAL_LENGTH_MM)
fov_y = calculate_fov(SENSOR_HEIGHT_MM,FOCAL_LENGTH_MM)

In [7]:
# Convert camera details into matrix form (roll, pitch and yaw are in radians)
def R_x(a):
    ca, sa = math.cos(a), math.sin(a)
    return np.array([
        [1, 0, 0],
        [0, ca, -sa],
        [0, sa,  ca]
    ])

def R_y(a):
    ca, sa = math.cos(a), math.sin(a)
    return np.array([
        [ ca, 0, sa],
        [  0, 1,  0],
        [-sa, 0, ca]
    ])

def R_z(a):
    ca, sa = math.cos(a), math.sin(a)
    return np.array([
        [ca, -sa, 0],
        [sa,  ca, 0],
        [ 0,   0, 1]
    ])

def get_camera_to_world_rotation(imu_roll, imu_pitch, imu_yaw, gimbal_pitch):
    
    # All angles in RADIANS
    # imu_     : drone body attitude (from MAVLink ATTITUDE)
    # gimbal_  : camera gimbal angles
    
    # Camera -> Gimbal
    # Camera optical axis (Z forward) →  Drone body forward (X)
    R_cam_gimbal = np.array([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1]
    ])

    # Gimbal -> Body 
    R_gimbal_body = (
        R_y(gimbal_pitch) 
    )

    # Body -> World 
    R_body_world = (
        R_z(imu_yaw) @
        R_y(imu_pitch) @
        R_x(imu_roll)
    )

    # Final rotation matrix
    return R_body_world @ R_gimbal_body @ R_cam_gimbal


In [8]:
# Used for phone testing
def get_geotag(
    bbox,
    frame_shape,
    drone_lat, drone_lon, drone_alt,
    imu_roll, imu_pitch, imu_yaw,
    gimbal_pitch
):
    x1, y1, x2, y2 = bbox
    u = (x1 + x2) * 0.5
    v = y2 - 0.15 * (y2 - y1)

    h, w = frame_shape[:2]
    cx, cy = w / 2, h / 2

    # Camera intrinsics from physical params 
    fx = (FOCAL_LENGTH_MM / SENSOR_WIDTH_MM) * w
    fy = (FOCAL_LENGTH_MM / SENSOR_HEIGHT_MM) * h

    # Pixel → camera ray 
    x_cam = (u - cx) / fx
    y_cam = -(v - cy) / fy   # IMPORTANT: flip Y axis
    z_cam = -1.0             # camera looking down

    ray_cam = np.array([x_cam, y_cam, z_cam])
    ray_cam /= np.linalg.norm(ray_cam)

    # Rotation matrices 
    roll  = math.radians(imu_roll)
    pitch = math.radians(imu_pitch + gimbal_pitch)
    yaw   = math.radians(imu_yaw)

    Rz = np.array([
        [ math.cos(yaw), -math.sin(yaw), 0],
        [ math.sin(yaw),  math.cos(yaw), 0],
        [ 0,              0,             1]
    ])

    Ry = np.array([
        [ math.cos(pitch), 0, math.sin(pitch)],
        [ 0,               1, 0],
        [-math.sin(pitch), 0, math.cos(pitch)]
    ])

    Rx = np.array([
        [1, 0,              0],
        [0, math.cos(roll), -math.sin(roll)],
        [0, math.sin(roll),  math.cos(roll)]
    ])

    R = Rz @ Ry @ Rx
    ray_world = R @ ray_cam

    # Ground intersection 
    if ray_world[2] >= -1e-6:
        return None

    t = drone_alt / -ray_world[2]
    east  = t * ray_world[0]
    north = t * ray_world[1]

    # ENU → GPS 
    dlat = north / 111111.0
    dlon = east / (111111.0 * math.cos(math.radians(drone_lat)))

    return drone_lat + dlat, drone_lon + dlon


In [9]:
def drop(lat, lon):
    print(f"🚑 DROPPING PARCEL at {lat}, {lon}")



def surveillance():
    print("🛰️ SURVEILLANCE MODE")
    


In [10]:
class KalmanGPS:
    def __init__(self, lat, lon):
        self.x = np.array([[lat], [lon]])
        self.P = np.eye(2) * 1e-4
        self.Q = np.eye(2) * 1e-6
        self.R = np.eye(2) * 1e-5

    def update(self, lat, lon):
        z = np.array([[lat], [lon]])
        self.P = self.P + self.Q
        K = self.P @ np.linalg.inv(self.P + self.R)
        self.x = self.x + K @ (z - self.x)
        self.P = (np.eye(2) - K) @ self.P
        return float(self.x[0]), float(self.x[1])


In [11]:
PERSON_CLASS_ID = 0
CONF_TH = 0.50
FRAME_CONFIRM = 10         
SERVE_RADIUS = 1.2       
SAME_PERSON_RADIUS = 1.2   
AVG_WINDOW = 10 
      

rtsp_url = "rtsp://192.168.144.25:8554/main.264"
# rtsp_url = r"F:\December\JP\ML\rtsp_record_20260106_213116.mp4"
cap = cv2.VideoCapture(rtsp_url, cv2.CAP_FFMPEG)

# cap = cv2.VideoCapture(r"E:\NIDAR\test\final\WhatsApp Video 2025-12-17 at 12.29.52.mp4")
model = YOLO(r"/home/stealthx/Desktop/Nidar_codebase/best.pt")

served_targets = []            
candidate_tracks = {}          
next_candidate_id = 1
csv_buffer = []



In [ ]:
def is_served(lat, lon):
    return any(
        geodesic((lat, lon), s).meters < SERVE_RADIUS
        for s in served_targets
    )

: 

In [ ]:


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=CONF_TH, verbose=False)[0]

    for box in results.boxes:
        if int(box.cls[0]) != PERSON_CLASS_ID:
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])

        gps = get_geotag(
            bbox=(x1, y1, x2, y2),
            frame_shape=frame.shape,
            drone_lat=drone_lat,
            drone_lon=drone_lon,
            drone_alt=drone_alt,
            imu_roll=imu_roll,
            imu_pitch=imu_pitch,
            imu_yaw=imu_yaw,
            gimbal_pitch=gimbal_pitch
        )

        if gps is None:
            continue

        lat, lon = gps

        if is_served(lat, lon):
            continue

        # ===== TRACK MATCHING =====
        matched_id = None
        for cid, track in candidate_tracks.items():
            if track["confirmed"]:
                continue
            d = geodesic((lat, lon), (track["lat"], track["lon"])).meters
            if d < SAME_PERSON_RADIUS:
                matched_id = cid
                break

        # ===== NEW PERSON =====
        if matched_id is None:
            candidate_tracks[next_candidate_id] = {
                "lat": lat,
                "lon": lon,
                "count": 1,
                "kalman": KalmanGPS(lat, lon),
                "lat_buf": deque([lat], maxlen=AVG_WINDOW),
                "lon_buf": deque([lon], maxlen=AVG_WINDOW),
                "confirmed": False
            }
            print(f"[NEW] Person ID {next_candidate_id}")
            matched_id = next_candidate_id
            next_candidate_id += 1

        # ===== EXISTING PERSON =====
        else:
            track = candidate_tracks[matched_id]
            track["count"] += 1

            filt_lat, filt_lon = track["kalman"].update(lat, lon)
            track["lat"] = filt_lat
            track["lon"] = filt_lon

            track["lat_buf"].append(filt_lat)
            track["lon_buf"].append(filt_lon)

            print(f"[TRACK] ID {matched_id} count={track['count']}")

            # ===== CONFIRM PERSON =====
            if track["count"] >= FRAME_CONFIRM and not track["confirmed"]:
                avg_lat = float(np.median(track["lat_buf"]))
                avg_lon = float(np.median(track["lon_buf"]))

                print(
                    f"🎯 DROP TARGET ID {matched_id}: "
                    f"{avg_lat:.6f}, {avg_lon:.6f}"
                )

                drop(avg_lat, avg_lon)
                served_targets.append((avg_lat, avg_lon))
                track["confirmed"] = True

                csv_buffer.append([
                    matched_id,
                    avg_lat,
                    avg_lon,
                    time.strftime("%Y-%m-%d %H:%M:%S")
                ])

        # ===== DRAW =====
        color = (0, 255, 0) if not candidate_tracks[matched_id]["confirmed"] else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"ID {matched_id}",
                    (x1, y1 - 6),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    cv2.imshow("Disaster Drone", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# ===== SAVE CSV =====
with open("geotagged_person_locations.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["person_id", "latitude", "longitude", "timestamp"])
    writer.writerows(csv_buffer)

cap.release()
cv2.destroyAllWindows()
print("🛑 SYSTEM STOPPED")


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/stealthx/.local/lib/python3.10/site-packages/cv2/qt/plugins"
